In [1]:
import numpy as np
import pandas as pd
from plotting import Plotting
from loading_preparing_data import PrepData
from gng_statistics import GNGstats
import statsmodels.formula.api as smf
import scipy

/Users/jolandamalamud/opt/miniconda3/envs/pandaenv/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# Set the file path
filepath = '../../'

# Create an instance of the PrepData class
prepdata = PrepData('panda', filepath, ['gad', 'phq', 'bdi'])
# Create an instance of the GNGstats class
gngstats = GNGstats()
# Create an instance of the Plotting class
plotting = Plotting('panda')

# Load raw task data
D = prepdata.load_data()
# Prepare action choices for plotting
data = prepdata.extract_data(D)
# load model fits
modelling = prepdata.load_modelfits()

# load rct data
dfRCT = prepdata.load_rctdata()

# Set the file path for saving figures related to the paper
figpath = filepath + 'results/paper_figures/'

In [3]:
# Define all variables of interest from RCT
baseline_vars = ['site', 'cis', 'dep', 'age', 'education', 'AD_past', \
               'sex', 'ethnic', 'fin', 'empstat', 'marstat', 'cisscore']

# Questionnaire variables which need to be log-transformed
log_transfer = [i + '0' for i in prepdata.psychiatric_questionnaire] + [i for i in dfRCT.columns if 'anh' in i]

# Create DataFrame
df_panda = prepdata.create_df(data, modelling, dfRCT, log_transfer)
df_panda = df_panda.astype(float)

In [4]:
# exclusion due to modelling
print(str(sum(modelling['most_parsimonious']['excluding'] == 0)) + ' included, ' + \
      str(sum(modelling['most_parsimonious']['excluding'] == 1)) + ' excluded')

890 included, 751 excluded


In [5]:
# ids in modelling but not in RCT bc never randomized or excluded due to not completing enough baseline measures
for i in data['subids']:
    if i not in np.array(dfRCT['ID']):
        print(i, data['sess'][data['subids']==i],modelling['most_parsimonious']['excluding'][data['subids']==i])

10025.0 [1.] [False]
20001.0 [1.] [False]
20227.0 [1.] [ True]
20256.0 [1.] [False]
30022.0 [1.] [ True]
40139.0 [1.] [False]


In [6]:
# Zscore all variables
df_zscored = pd.DataFrame()
zscored_columns = [col for col in df_panda.columns if col not in ['ID', 'subids', 'group']]
df_zscored = pd.concat([(df_panda[col] - df_panda[col].mean()) / df_panda[col].std(ddof=0) \
                    for col in zscored_columns], axis=1)
df_zscored.columns = [i + '_zscore' for i in df_zscored.columns]
df_panda = pd.concat([df_panda, df_zscored], axis=1)

In [7]:
# Number for patients
n_gng = len(np.unique(data['subids']))
n_rct = len(dfRCT['ID'])
overlap = np.intersect1d(np.unique(data['subids']), np.array(dfRCT['ID']))
print(['#GNG: ' + str(len(np.unique(data['subids']))), '#RCT ' + str(len(df_panda))])
print(str(n_rct - len(overlap)) + ' patients participated in the RCT but there is no task data')
print(str(n_gng - len(overlap)) + ' patients provided task data but not in randomized')
print(str(len(overlap)) + ' total number of patients with at least one task run')

['#GNG: 635', '#RCT 653']
25 patients participated in the RCT but there is no task data
6 patients provided task data but not in randomized
629 total number of patients with at least one task run


---------------------------------------------------------------------------------------------------------------
Exclusion
--

In [8]:
# number of excluded subjects due to not performing the task properly
for t in range(3):
    no_data = df_panda['exclusion' + str(t)].isna()
    ex = df_panda['exclusion' + str(t)] == 1
    df_panda.loc[~no_data, 'exclusion' + str(t)] = (ex[~no_data]).astype(int)

n_subject = []
exclusion = []
for t in range(3):
    exclusion.append(np.nansum(df_panda['exclusion' + str(t)]))
    n_subject.append(sum(~df_panda['exclusion' + str(t)].isna()))
    print('# of excluded subjects at week ' + str(prepdata.fu_weeks[t]) + ':\t' + str(exclusion[t]) + \
      ' (' + str(round(exclusion[t]/n_subject[t],2)) +'%)' + \
         ', included: ' + str(round(n_subject[t]-exclusion[t],2)))
print('total # of excluded subjects:\t\t' + str(sum(exclusion)) + \
      ' (' + str(round(sum(exclusion)/(sum(n_subject))*100)) +'%)')

# of excluded subjects at week 0:	316.0 (0.51%), included: 305.0
# of excluded subjects at week 2:	230.0 (0.43%), included: 299.0
# of excluded subjects at week 6:	201.0 (0.42%), included: 282.0
total # of excluded subjects:		747.0 (46%)


In [9]:
for t in range(3):
    for i in range(2):
        print('T = ' + str(t) + ' group ' + str(i) + ': tot=' + \
              str(sum(~df_panda['exclusion' + str(t)][df_panda['group']==i].isna())) + \
              ' in=' + str((df_panda['exclusion' + str(t)][df_panda['group']==i]==0).sum()) + \
              ' ex=' + str(df_panda['exclusion' + str(t)][df_panda['group']==i].sum()))

T = 0 group 0: tot=310 in=158 ex=152.0
T = 0 group 1: tot=311 in=147 ex=164.0
T = 1 group 0: tot=266 in=166 ex=100.0
T = 1 group 1: tot=263 in=133 ex=130.0
T = 2 group 0: tot=247 in=144 ex=103.0
T = 2 group 1: tot=236 in=138 ex=98.0


In [10]:
included = df_panda[['exclusion0', 'exclusion1', 'exclusion2']] == 0
print('included task runs: ' + str(included.values.sum()))
print(str((included.sum(axis=1) > 0).sum()) + ' ('+ str(round((included.sum(axis=1) > 0).sum()/ 655 * 100)) +  '% of those randomised)')

included task runs: 886
435 (66% of those randomised)


In [11]:
# chi2 pearson test to compare included vs excluded in sertraline vs placebo
for t in range(1,3):
    tmp = gngstats.chi2_test(df_panda['exclusion' + str(t)], df_panda['group'], ['sertraline', 'placebo'])
    tmp.insert(0, 'variable', 'exclusion')
    display(tmp)
    print('excluded at T = ' + str(t) + ' (N = ' + str(sum(~df_panda['exclusion' + str(t)].isna() & ~df_panda['group'].isna())) + \
          '): ' + str(np.round((df_panda['group'][df_panda['exclusion' + str(t)]==1] == 1).mean()*100)) \
          + '% sertraline, ' + str(np.round((df_panda['group'][df_panda['exclusion' + str(t)]==1] == 0).mean()*100)) \
          + '% placebo')

,variable,sertraline,placebo,X2,p
exclusion1,,,,,
0.0,exclusion,133,166,7.064,0.008
1.0,exclusion,130,100,7.064,0.008


excluded at T = 1 (N = 529): 57.0% sertraline, 43.0% placebo


,variable,sertraline,placebo,X2,p
exclusion2,,,,,
0.0,exclusion,138,144,0.0,1.0
1.0,exclusion,98,103,0.0,1.0


excluded at T = 2 (N = 483): 49.0% sertraline, 51.0% placebo


In [12]:
for t in range(3):
    gngstats.glm('iL' + str(t) + ' ~ group', df_panda, ['group'])

group: 	beta: 1.93,	CI: [-0.18,4.04],	pvalue: 0.0736
group: 	beta: -0.13,	CI: [-2.51,2.25],	pvalue: 0.914
group: 	beta: -0.09,	CI: [-2.79,2.6],	pvalue: 0.9459


In [13]:
# is exclusion related to baseline variables? Logistic regression
baseline_continuous =  ['age_zscore', 'gad0log_zscore', 'phq0log_zscore', 'bdi0log_zscore']
baseline_categorical = ['site', 'cis', 'dep', 'education', \
                        'AD_past','sex', 'ethnic', 'fin', 'empstat', 'marstat', 'group']
tab_list = []
tab_list_reduced = []
for i in range(3):
    tab = []
    for j in baseline_continuous + baseline_categorical:
        model = smf.logit('exclusion' + str(i) +' ~ ' + j, data=df_panda).fit(disp=False);
        tab.append([j, round(model.params[1],2), round(model.pvalues[1],3)])
    tab_list.append(pd.DataFrame(tab, columns=['baseline variable', 'estimate', 'pvalue']))
for i in tab_list:
    tab_list_reduced.append(i[i['pvalue']<0.05])
print('baseline variables related to non-informative task runs?')
plotting.display_side_by_side(tab_list_reduced[0],tab_list_reduced[1],tab_list_reduced[2])
# print('multiple comparisons: ' + str(round(0.05/(3*len(baseline_continuous + baseline_categorical)),4)))

baseline variables related to non-informative task runs?


,baseline variable,estimate,pvalue
0,age_zscore,0.67,0.000
7,education,0.71,0.000
8,AD_past,0.55,0.001
,baseline variable,estimate,pvalue
0,age_zscore,0.87,0.000
7,education,0.92,0.000
8,AD_past,0.61,0.001
14,group,0.48,0.006
,baseline variable,estimate,pvalue
0,age_zscore,0.85,0.000


All time points, the more non-informative data:
- the older.
- the lower highest educational qualification (1=A Level or higher;2=GCSE,standard grade or other;3=no formal qualification)

At week 2, the more non-informative data:
- the more antidepressants in the past (1=no;2=yes).
- the more in sertraline group (0=placebo;1=sertraline).

At week 6, the more non-informative data:
- the higher GAD-7 score.
- the higher CISR-RCIS-R total score (1=0-11, 2=12-19; 3=>=2-49)
- the more non-employment status (1=in paid employment;2=not employed)

In [14]:
for i in range(3):
    print('education covariate at week ' + ['0','2','6'][i])
    educationtest = gngstats.chi2_test(df_panda['education'], df_panda['exclusion' + str(i)], \
                                  ['non-informative data', 'informative data'])
    educationtest.index = ['A Level&higher','GCSE, standard grade&other','no formal qualification']
    display(educationtest)

i == 2
print('cisr covariate at week 6')
cistest = gngstats.chi2_test(df_panda['cis'], df_panda['exclusion' + str(i)], \
                                  ['non-informative data', 'informative data'])
cistest.index = ['0-11', '12-19', '>=20']
display(cistest)

education covariate at week 0


,non-informative data,informative data,X2,p
A Level&higher,199,234,26.758,0.0
"GCSE, standard grade&other",89,69,26.758,0.0
no formal qualification,27,2,26.758,0.0


education covariate at week 2


,non-informative data,informative data,X2,p
A Level&higher,134,236,31.738,0.0
"GCSE, standard grade&other",78,58,31.738,0.0
no formal qualification,18,4,31.738,0.0


education covariate at week 6


,non-informative data,informative data,X2,p
A Level&higher,110,227,43.62,0.0
"GCSE, standard grade&other",72,53,43.62,0.0
no formal qualification,18,2,43.62,0.0


cisr covariate at week 6


,non-informative data,informative data,X2,p
0-11,87,164,10.132,0.006
12-19,65,66,10.132,0.006
>=20,48,52,10.132,0.006


Summary:
--
- more patients not doing the task properly in the sertraline group at week 2 (follow-up 1)
- patients not doing the task properly were older, less educated, and were taking antidepressants in the past over all sessions --> control for age, education, and AD in the past
- at week 6 exclusion additionally related to baseline gad and employment status

In [15]:
# define covariates for further analyses
stratification_covariates = '+ site + dep + cis'
exclusion_covariates = '+ age + education + AD_past + empstat'
missing_covariates = '+ cisscore + gad0log + phq0log + fin + ethnic'

---------------------------------------------------------------------------------------------------------------
Included sample
--

In [16]:
# demographics
df_panda['ethnic'] = df_panda['ethnic'].replace([2, 3, 4, 5, 6], 2)
ex = (df_panda['exclusion0'] == 0) | (df_panda['exclusion1'] == 0) | (df_panda['exclusion2'] == 0)
group_label = ['placebo', 'sertraline', 'overall']
demographic_variables1 = ['age', 'gad0', 'phq0', 'bdi0', 'cisscore']
demographic_variables2 = ['site', 'cis', 'dep', 'education', 'AD_past', 'sex', \
                         'ethnic', 'fin', 'empstat', 'marstat']
T = np.empty([100,4], dtype=object)
T[0,3] = 'pvalue'
                          
for dd, d in enumerate(demographic_variables1):
    T[dd+1,0] = d
    model = smf.logit('group ~ ' + d, data=df_panda[ex]).fit(disp=False);
    T[dd+1,3] = round(model.pvalues[1],3)
#     T[dd+1,3] = pstats.group_ttest(df_panda[d][ex], df_panda['group'][ex], \
#                                    ['sertraline', 'placebo'])['p'].iloc[0]
    for g in range(2):
        T[dd+1,g+1] = str(np.round(df_panda[d][(df_panda['group']==g)&ex].mean(),2)) + ' (' + \
                    str(np.round(df_panda[d][(df_panda['group']==g)&ex].std(),2)) + ')'
        
for g in range(2):
    T[0,g+1] = group_label[g] + ' (N = ' + str(sum((df_panda['group']==g)&ex)) + ')'
    dd = len(demographic_variables1)+1
    for d in demographic_variables2:
        for i in range(int(max(df_panda[d][ex]))):
            T[dd,0] = d + str(i)
            model = smf.logit('group ~ ' + d, data=df_panda[ex]).fit(disp=False);
            T[dd,3] = round(model.pvalues[1],3)
#             T[dd,3] = pstats.chi2_test(df_panda[d][ex], df_panda['group'][ex], \
#                                        ['sertraline', 'placebo'])['p'].iloc[0]
            T[dd,g+1] = str(sum((df_panda[d][ex]==i+1) & (df_panda['group'][ex]==g))) + \
            ' (' + str(round(sum((df_panda[d][ex]==i+1) & (df_panda['group'][ex]==g))/ \
                             sum((df_panda['group'][ex]==g))*100)) + '%)'
            dd += 1
T = T[~(T == None).all(axis=1),:]

In [17]:
# print demographics table in latex
from tabulate import tabulate
print(tabulate(T, headers='firstrow', tablefmt='latex'))

\begin{tabular}{lllr}
\hline
 None       & placebo (N = 221)   & sertraline (N = 214)   &   pvalue \\
\hline
 age        & 36.03 (12.97)       & 36.84 (14.29)          &    0.535 \\
 gad0       & 9.6 (5.11)          & 9.27 (5.19)            &    0.496 \\
 phq0       & 12.47 (5.66)        & 11.71 (5.77)           &    0.171 \\
 bdi0       & 24.19 (9.9)         & 24.06 (10.17)          &    0.889 \\
 cisscore   & 11.0 (4.8)          & 10.4 (4.82)            &    0.201 \\
 site0      & 98 (44\%)            & 92 (43\%)               &    0.681 \\
 site1      & 39 (18\%)            & 37 (17\%)               &    0.681 \\
 site2      & 48 (22\%)            & 47 (22\%)               &    0.681 \\
 site3      & 36 (16\%)            & 38 (18\%)               &    0.681 \\
 cis0       & 37 (17\%)            & 45 (21\%)               &    0.389 \\
 cis1       & 58 (26\%)            & 51 (24\%)               &    0.389 \\
 cis2       & 126 (57\%)           & 117 (55\%)              &    0.389 \\
 

In [18]:
# Baseline GNG parameters related to psychiatric scores?
print('multiple comparison: p < ' + str(np.round(0.05/(8*3),3)))
for j in [i + '0' for i in prepdata.psychiatric_questionnaire]:
    print(j)
    print(100*'-')
    for p in prepdata.parameter_labels[:-1]:
        gngstats.glm(j + ' ~' + p + '0', df_panda[df_panda['exclusion0'] == 0], \
                       [p + '0'])
    print(100*'-')

multiple comparison: p < 0.002
gad0
----------------------------------------------------------------------------------------------------
rew_se0: 	beta: 0.4,	CI: [-0.18,0.98],	pvalue: 0.1791
loss_se0: 	beta: -0.25,	CI: [-0.86,0.36],	pvalue: 0.425
rew_LR0: 	beta: -0.07,	CI: [-0.59,0.45],	pvalue: 0.7906
loss_LR0: 	beta: 0.23,	CI: [-0.11,0.58],	pvalue: 0.1839
app_Pav0: 	beta: -0.09,	CI: [-1.13,0.95],	pvalue: 0.8642
av_Pav0: 	beta: 0.04,	CI: [-0.67,0.76],	pvalue: 0.9056
noise0: 	beta: -0.08,	CI: [-0.56,0.4],	pvalue: 0.7501
bias0: 	beta: -0.01,	CI: [-0.63,0.62],	pvalue: 0.9876
----------------------------------------------------------------------------------------------------
phq0
----------------------------------------------------------------------------------------------------
rew_se0: 	beta: 0.43,	CI: [-0.23,1.09],	pvalue: 0.198
loss_se0: 	beta: -0.23,	CI: [-0.92,0.47],	pvalue: 0.5207
rew_LR0: 	beta: -0.11,	CI: [-0.71,0.48],	pvalue: 0.7069
loss_LR0: 	beta: 0.25,	CI: [-0.14,0.64],	pvalue

---------------------------------------------------------------------------------------------------------------
MIXED EFFECTS MODELLING
--------------------------------------------------------------------------------------------------------

In [19]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [20]:
# create dataframe for mixed-effects modelling
mle_df = prepdata.create_mle_df(df_panda, prepdata.gng_variables + prepdata.parameter_labels, baseline_vars)
mle_df['anhlog'] = np.hstack((df_panda['anh0'],df_panda['anh1'],df_panda['anh2']))
for i in mle_df.columns:
    mle_df[i + '_zscore'] = gngstats.normalize(mle_df[i])
    
included_subjects = mle_df['exclusion']==0
timing = [mle_df['time']<2, (mle_df['time']==0)|(mle_df['time']==2), mle_df['time']<3]
timing_label = ['2','6','over time', 'time x group']

Preregistered Hypotheses
--

In [21]:
# H1: aversive Pav related to sertraline?
p = 'av_Pav'
timing_label = ['2','6','over time', 'time x group']
for i in range(2):
    print('T = ' + timing_label[i], end = ', ')
    tmp = mle_df[timing[i]&included_subjects]
    for g in range(2):
        print(['placebo', 'sertraline'][g] + ': ' + \
              str(np.round(np.nanmean(tmp[p][(tmp['group']==g)&(tmp['time']==i+1)]),2)) + '±' + \
              str(np.round(np.nanstd(tmp[p][(tmp['group']==g)&(tmp['time']==i+1)]),2)), end=', ')
    gngstats.mle(p + ' ~ group + time' + stratification_covariates + exclusion_covariates, tmp, ['group'],[])

for i in range(2):
    print(timing_label[i+2], end = ', ')
    gngstats.mle(p + ' ~ ' + ['group + time', 'group * time'][i] + stratification_covariates + \
                 exclusion_covariates, mle_df[timing[2]&included_subjects], [['group', 'group:time'][i]],[])


T = 2, placebo: -0.55±0.79, sertraline: -0.71±0.79, group: 	beta: -0.12,	CI: [-0.29,0.05],	pvalue: 0.1662
T = 6, placebo: -0.77±0.85, sertraline: -0.7±0.83, group: 	beta: 0.12,	CI: [-0.06,0.3],	pvalue: 0.2058
over time, group: 	beta: -0.01,	CI: [-0.14,0.12],	pvalue: 0.8931
time x group, group:time: 	beta: 0.12,	CI: [-0.05,0.29],	pvalue: 0.1592


In [22]:
# Pavlovian bias at baseline
p = 'av_Pav'
for g in range(2):
    print(df_panda[p + '0'][(df_panda['exclusion0']==0)&(df_panda['group']==g)].mean(), \
          df_panda[p + '0'][(df_panda['exclusion0']==0)&(df_panda['group']==g)].std())

-0.5026456524007136 0.8287859100454065
-0.5538415294603797 0.7896871772382293


In [23]:
# H2: aversive Pav related to anxiety?
p = 'av_Pav'
for i in range(3):
    tmp = mle_df[timing[i]&included_subjects]
    print(timing_label[i] + ':', end='\t')
    gngstats.mle('gadlog ~ ' + p + ' + group + time' + stratification_covariates + exclusion_covariates, \
               tmp, [p], p)

2:	av_Pav: 	beta: -0.01,	CI: [-0.03,0.01],	pvalue: 0.406
6:	model not converged -> random effects removed:
av_Pav: 	beta: -0.02,	CI: [-0.05,0.01],	pvalue: 0.125
over time:	av_Pav: 	beta: -0.02,	CI: [-0.04,0.0],	pvalue: 0.0599


In [24]:
# H4: can baseline aversive Pav predict anxiety at week 12?
p = 'av_Pav0'
gngstats.glm('gad3log ~ ' + p + ' + gad0log + group ' + stratification_covariates + exclusion_covariates, \
           df_panda[df_panda['exclusion0']==0].astype(float), [p]);

av_Pav0: 	beta: -0.02,	CI: [-0.07,0.03],	pvalue: 0.4576


In [25]:
# H5: appetitive Pav related to anxiety?
p = 'app_Pav'
for i in range(3):
    tmp = mle_df[timing[i]&included_subjects]
    print(timing_label[i] + ':', end='\t')
    gngstats.mle('phqlog ~ ' + p + ' + group + time' + stratification_covariates + exclusion_covariates, \
               tmp, [p], p)
    

2:	app_Pav: 	beta: -0.01,	CI: [-0.04,0.02],	pvalue: 0.3771
6:	

/Users/jolandamalamud/opt/miniconda3/envs/pandaenv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/jolandamalamud/opt/miniconda3/envs/pandaenv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


app_Pav: 	beta: -0.03,	CI: [-0.07,0.0],	pvalue: 0.0582
over time:	app_Pav: 	beta: -0.03,	CI: [-0.05,0.0],	pvalue: 0.0661


In [26]:
# H6: reward sensitivity related to anhedonia?
p = 'rew_se'
for i in range(3):
    tmp = mle_df[timing[i]&included_subjects]
    print(timing_label[i] + ':', end='\t')
    gngstats.mle('anhlog ~ ' + p + ' + group + time' + stratification_covariates + exclusion_covariates, \
        tmp, [p], p)


2:	

/Users/jolandamalamud/opt/miniconda3/envs/pandaenv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


rew_se: 	beta: -0.01,	CI: [-0.08,0.05],	pvalue: 0.6458
6:	rew_se: 	beta: 0.04,	CI: [-0.02,0.11],	pvalue: 0.1918
over time:	rew_se: 	beta: 0.0,	CI: [-0.05,0.05],	pvalue: 0.9071


Exploratory Analyses
--

In [27]:
# Create a longer DF including the week 12 measurment
symptom_df = mle_df[['subject','group', 'time', 'exclusion'] + baseline_vars + \
                    [i for i in mle_df if i[:3] in prepdata.psychiatric_questionnaire]]
keys = [i + '3log' for i in prepdata.psychiatric_questionnaire]
values = [i + 'log' for i in prepdata.psychiatric_questionnaire]
result_dict = dict(zip(keys, values))
tmp = df_panda[['group'] + baseline_vars + \
               [i + '3log' for i in prepdata.psychiatric_questionnaire]].rename(columns=result_dict)
tmp['subject'] = range(len(tmp))
tmp['time'] = 3
tmp['exclusion'] = 0
symptom_df = pd.concat([symptom_df, tmp], ignore_index=True)

In [28]:
# Effect of sertraline on symptoms in included sample at week 2, 6 and 12 separately and over time
for p in prepdata.psychiatric_questionnaire:
    for i in range(3):
        tmp = symptom_df[(symptom_df['exclusion']==0)&((symptom_df['time']==0)|(symptom_df['time']==i+1))]
        print(p + ' -> T = ' + ['2', '6', '12'][i] + ':', end=' ')
        gngstats.mle(p + 'log ~ group + time' + stratification_covariates + exclusion_covariates, \
                   tmp, ['group'], 'group')
    tmp = symptom_df[symptom_df['exclusion']==0]
    print('over time:', end=' ' + p + ' ')
    gngstats.mle(p + 'log ~ group + time' + stratification_covariates + exclusion_covariates, \
               tmp, ['group'], 'group')
    print('group x time:', end=' ' + p + ' ')
    gngstats.mle(p + 'log ~ group * time' + stratification_covariates + exclusion_covariates, \
               tmp, ['group:time'], 'group')
    print(100*'-')

gad -> T = 2: group: 	beta: 0.0,	CI: [-0.06,0.06],	pvalue: 0.9949
gad -> T = 6: group: 	beta: -0.1,	CI: [-0.17,-0.03],	pvalue: 0.0042
gad -> T = 12: group: 	beta: -0.12,	CI: [-0.18,-0.07],	pvalue: 0.0
over time: gad group: 	beta: -0.08,	CI: [-0.13,-0.04],	pvalue: 0.0001
group x time: gad group:time: 	beta: -0.03,	CI: [-0.06,-0.01],	pvalue: 0.0147
----------------------------------------------------------------------------------------------------
phq -> T = 2: group: 	beta: 0.01,	CI: [-0.04,0.05],	pvalue: 0.804
phq -> T = 6: group: 	beta: -0.01,	CI: [-0.06,0.05],	pvalue: 0.8255
phq -> T = 12: group: 	beta: -0.07,	CI: [-0.12,-0.02],	pvalue: 0.0103
over time: phq group: 	beta: -0.03,	CI: [-0.06,0.01],	pvalue: 0.1662
group x time: phq group:time: 	beta: -0.03,	CI: [-0.06,-0.0],	pvalue: 0.0193
----------------------------------------------------------------------------------------------------
bdi -> T = 2: group: 	beta: 0.01,	CI: [-0.04,0.06],	pvalue: 0.5744
bdi -> T = 6: group: 	beta: 0.0,

Sertraline had no effect on depressive symptoms at week 2 and 6 and only small effect at week 12 in the limited sample, but we observed evidence that differences in GAD-7 scores
became larger over time (constant to complete sample and original trial paper)

Drug effects:
--
only at 2 weeks:
- loss LR HIGHER in sertraline group


In [29]:
print('correction for multiple comparison: pval < ' + str(0.05/8))

correction for multiple comparison: pval < 0.00625


In [30]:
# Drug effect on parameter
timing_label = ['2','6','over time', 'time x group']
for p in [i for i in prepdata.parameter_labels[:-1] if i != 'av_Pav']:
    print(p)
    for i in range(2):
        print('T = ' + ['2', '6', '12'][i], end = ', ')
        tmp = mle_df[timing[i]&(mle_df['exclusion']==0)]
        gngstats.mle(p + ' ~ group + time' + stratification_covariates + exclusion_covariates, tmp, ['group'],[])

    for i in range(2):
        print(timing_label[i+2], end = ', ')
        gngstats.mle(p + ' ~ ' + ['group + time', 'group * time'][i] + stratification_covariates + \
                     exclusion_covariates, mle_df[timing[2]&included_subjects], ['group'],[])
    
    print(100*'-')

rew_se
T = 2, group: 	beta: 0.21,	CI: [-0.01,0.44],	pvalue: 0.0666
T = 6, group: 	beta: -0.14,	CI: [-0.37,0.1],	pvalue: 0.2568
over time, group: 	beta: 0.01,	CI: [-0.14,0.17],	pvalue: 0.8535
time x group, group: 	beta: 0.19,	CI: [-0.2,0.58],	pvalue: 0.337
----------------------------------------------------------------------------------------------------
loss_se
T = 2, group: 	beta: -0.16,	CI: [-0.37,0.05],	pvalue: 0.1263
T = 6, group: 	beta: 0.13,	CI: [-0.09,0.34],	pvalue: 0.2584
over time, group: 	beta: 0.01,	CI: [-0.14,0.17],	pvalue: 0.8523
time x group, group: 	beta: 0.0,	CI: [-0.34,0.34],	pvalue: 0.9957
----------------------------------------------------------------------------------------------------
rew_LR
T = 2, group: 	beta: -0.06,	CI: [-0.32,0.2],	pvalue: 0.6752
T = 6, group: 	beta: 0.14,	CI: [-0.12,0.4],	pvalue: 0.2919
over time, group: 	beta: 0.08,	CI: [-0.11,0.26],	pvalue: 0.4266
time x group, group: 	beta: 0.08,	CI: [-0.36,0.52],	pvalue: 0.7233
--------------------------

Association with symptoms
--
loss LR is positively related to GAD at week 2, 6, and over all sessions

In [31]:
print('correction for multiple comparison: pval < ' + str(0.05/(8*3)))

correction for multiple comparison: pval < 0.0020833333333333333


In [32]:
# Parameter related to anxiety?
for j in prepdata.psychiatric_questionnaire[:-1]:
    print(j)
    for p in  [i for i in prepdata.parameter_labels[:-1] if i != 'av_Pav']:
        for i in range(3):
            tmp = mle_df[timing[i]&(mle_df['exclusion']==0)]
            print(timing_label[i] + ':', end='\t')
            gngstats.mle(j + 'log_zscore ~ ' + p + '_zscore + group + time' + stratification_covariates + exclusion_covariates, \
                       tmp, [p + '_zscore'], p + '_zscore')
        print(100*'-')
    print(100*'=')

gad
2:	rew_se_zscore: 	beta: -0.03,	CI: [-0.09,0.04],	pvalue: 0.4348
6:	rew_se_zscore: 	beta: 0.05,	CI: [-0.03,0.12],	pvalue: 0.2148
over time:	rew_se_zscore: 	beta: 0.0,	CI: [-0.05,0.06],	pvalue: 0.8858
----------------------------------------------------------------------------------------------------
2:	model not converged -> random effects removed:
loss_se_zscore: 	beta: -0.05,	CI: [-0.12,0.01],	pvalue: 0.102
6:	loss_se_zscore: 	beta: -0.04,	CI: [-0.12,0.03],	pvalue: 0.2362
over time:	loss_se_zscore: 	beta: -0.06,	CI: [-0.12,0.0],	pvalue: 0.0505
----------------------------------------------------------------------------------------------------
2:	rew_LR_zscore: 	beta: 0.01,	CI: [-0.05,0.07],	pvalue: 0.8283
6:	rew_LR_zscore: 	beta: -0.02,	CI: [-0.09,0.05],	pvalue: 0.5147
over time:	model not converged -> random effects removed:
rew_LR_zscore: 	beta: -0.01,	CI: [-0.06,0.05],	pvalue: 0.8133
----------------------------------------------------------------------------------------------

In [33]:
# Do cognitive parameters relate to baseline variables over all time points?
for i in prepdata.parameter_labels[:-1]:
    print(i, end=':\t')
    m = gngstats.mle(i + ' ~ group + time' + stratification_covariates + exclusion_covariates, \
                        mle_df[mle_df['exclusion']==0], [], [])
    mm = m.summary().tables[1][1:-1]
    sig = m.pvalues < 0.05/8
    if sig[1:-1].any(): display(mm[sig[1:-1]])
    else: print('none')
    

rew_se:	none
loss_se:	none
rew_LR:	

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
age,-0.011,0.003,-3.417,0.001,-0.018,-0.005


loss_LR:	none
app_Pav:	

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
time,-0.082,0.023,-3.594,0.000,-0.127,-0.037
age,0.011,0.002,6.484,0.000,0.008,0.014


av_Pav:	

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
time,-0.102,0.032,-3.228,0.001,-0.164,-0.040
age,0.012,0.003,4.742,0.000,0.007,0.017


noise:	none
bias:	

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
time,0.131,0.039,3.371,0.001,0.055,0.208
age,-0.026,0.003,-8.999,0.000,-0.032,-0.021


- app and av Pav decrease over time
- Note: reward learning rate, Pavlovian biases, and go bias are related to age!

In [34]:
# Association between baseline gad and loss LR?
gngstats.glm('gad0log ~ loss_LR0' + stratification_covariates + exclusion_covariates, \
                            df_panda[df_panda['exclusion0']==0], ['loss_LR0'])

loss_LR0: 	beta: 0.01,	CI: [-0.0,0.02],	pvalue: 0.2393


In [35]:
# Association between gad at week 6 and loss LR at week 2?
gngstats.glm('gad2log ~ loss_LR1' + stratification_covariates + exclusion_covariates, \
                            df_panda[df_panda['exclusion1']==0], ['loss_LR1'])

loss_LR1: 	beta: 0.01,	CI: [-0.02,0.04],	pvalue: 0.4726


---------------------------------------------------------------------------------------------------------------
Change in parameter
--

In [36]:
# Compute change in parameter estimates
df_panda = prepdata.parameter_change(df_panda)

In [37]:
# number of patients with early change
print('N = ' + str(sum((df_panda['exclusion0']==0)&(df_panda['exclusion1']==0))))
print('N = ' + str(sum((df_panda['exclusion1']==0)&(df_panda['exclusion2']==0))))

N = 213
N = 206


In [38]:
# Drug effect on change in aversive Pavlovian bias
changes = ['01', '12']
change_labels = ['baseline and week 2', 'week 2 and week 6']
group_label = ['placebo___','sertraline']
p = 'av_Pav'
for i in range(len(changes)):
    print('av Pav change between ' + change_labels[i] + ': ', end='\t')
    gngstats.glm(p + '_slope' + changes[i] + ' ~ group' + stratification_covariates + exclusion_covariates, \
                            df_panda[(df_panda[['exclusion' + changes[i][0],\
                                                'exclusion' + changes[i][1]]] == 0).all(axis=1)], ['group'])
    for g in range(2):
        print(group_label[g],end=':\t')
        print(scipy.stats.ttest_1samp(df_panda[p + '_slope' + changes[i]][(df_panda[['exclusion' + changes[i][0], \
                                                                      'exclusion' + changes[i][1]]] == 0).all(axis=1) \
                                                       &(df_panda['group']==g)], popmean=0))

av Pav change between baseline and week 2: 	group: 	beta: -0.08,	CI: [-0.32,0.15],	pvalue: 0.4846
placebo___:	TtestResult(statistic=-1.5174922148422907, pvalue=0.13181705494817714, df=118)
sertraline:	TtestResult(statistic=-2.042845265728973, pvalue=0.04389787342236844, df=93)
av Pav change between week 2 and week 6: 	group: 	beta: 0.1,	CI: [-0.14,0.35],	pvalue: 0.4116
placebo___:	TtestResult(statistic=-2.066623802688763, pvalue=0.041034711661585364, df=114)
sertraline:	TtestResult(statistic=-0.5799026681607216, pvalue=0.5634292127487901, df=90)


In [39]:
# Early changes in aversive Pav predicting treatment outcome?
p = 'av_Pav'
for j in prepdata.psychiatric_questionnaire:
    print('#' * 100)
    print(j + ':')
    gngstats.glm(j + '3log ~' + p + '_slope01 + ' + j + '0log + group' + stratification_covariates \
               + exclusion_covariates, df_panda[(df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
               [p + '_slope01'])
    gngstats.glm(j + '3log ~' + p + '_slope01* group +' +  j + '0log' + stratification_covariates \
               + exclusion_covariates, df_panda[(df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
               [p + '_slope01:group'])
print('-' * 100)
for i in range(2):
    print(group_label[i], end=': ')
    gngstats.glm(j + '3log ~' + p + '_slope01 +' +  j + '0log' + stratification_covariates \
               + exclusion_covariates, df_panda[(df_panda['group'] == i) & \
                                                (df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
               [p + '_slope01'])   

####################################################################################################
gad:
av_Pav_slope01: 	beta: 0.02,	CI: [-0.03,0.08],	pvalue: 0.418
av_Pav_slope01:group: 	beta: 0.04,	CI: [-0.07,0.16],	pvalue: 0.4581
####################################################################################################
phq:
av_Pav_slope01: 	beta: 0.06,	CI: [0.0,0.11],	pvalue: 0.04
av_Pav_slope01:group: 	beta: 0.07,	CI: [-0.04,0.19],	pvalue: 0.1886
####################################################################################################
bdi:
av_Pav_slope01: 	beta: 0.07,	CI: [0.01,0.13],	pvalue: 0.0177
av_Pav_slope01:group: 	beta: 0.13,	CI: [0.01,0.25],	pvalue: 0.0313
----------------------------------------------------------------------------------------------------
placebo___: av_Pav_slope01: 	beta: 0.01,	CI: [-0.08,0.09],	pvalue: 0.8988
sertraline: av_Pav_slope01: 	beta: 0.14,	CI: [0.05,0.23],	pvalue: 0.002


In [40]:
# Drug effect on change in loss learning rate
changes = ['01', '12']
change_labels = ['baseline and week 2', 'week 2 and week 6']
group_label = ['placebo___','sertraline']
p = 'loss_LR'
for i in range(len(changes)):
    print('loss LR change between ' + change_labels[i] + ': ', end='\t')
    gngstats.glm(p + '_slope' + changes[i] + ' ~ group' + stratification_covariates + exclusion_covariates, \
                            df_panda[(df_panda[['exclusion' + changes[i][0],\
                                                'exclusion' + changes[i][1]]] == 0).all(axis=1)], ['group'])
    for g in range(2):
        print(group_label[g],end=':\t')
        print(scipy.stats.ttest_1samp(df_panda[p + '_slope' + changes[i]][(df_panda[['exclusion' + changes[i][0], \
                                                                      'exclusion' + changes[i][1]]] == 0).all(axis=1) \
                                                       &(df_panda['group']==g)], popmean=0))

loss LR change between baseline and week 2: 	group: 	beta: 0.75,	CI: [0.18,1.31],	pvalue: 0.0092
placebo___:	TtestResult(statistic=-0.7038282863694532, pvalue=0.4829262849798154, df=118)
sertraline:	TtestResult(statistic=2.739613117919031, pvalue=0.007373976818790651, df=93)
loss LR change between week 2 and week 6: 	group: 	beta: -0.72,	CI: [-1.27,-0.17],	pvalue: 0.0108
placebo___:	TtestResult(statistic=3.4427276226646897, pvalue=0.0008061888957859322, df=114)
sertraline:	TtestResult(statistic=-0.3191464671381358, pvalue=0.7503550084884072, df=90)


In [41]:
# Association between gad at week 6 and change in loss LR between baseline and week 2?
gngstats.glm('gad2log ~ loss_LR_slope01' + stratification_covariates + exclusion_covariates, \
                            df_panda[(df_panda[['exclusion0','exclusion1']]==0).all(axis=1)], ['loss_LR_slope01'])

loss_LR_slope01: 	beta: 0.0,	CI: [-0.02,0.03],	pvalue: 0.7655


Replicate in whole sample
--

In [42]:
def convert_stats_table(table):
    results_as_html = table.summary().tables[1].as_html()
    x = pd.read_html(results_as_html, header=0, index_col=0)[0]
    return x.iloc[1].values

In [62]:
whole_sample_df = df_panda.copy()
for i in range(3): whole_sample_df['exclusion' + str(i)] = 0
whole_sample_mle_df = mle_df.copy()
whole_sample_mle_df['exclusion'] = 0

gngstats.calculate_exploratory_stats(whole_sample_mle_df, whole_sample_df, np.ones(len(df_panda)), \
                                            stratification_covariates)


1: drug effect on loss learning rate at week 2
group: 	beta: 0.32,	CI: [0.03,0.61],	pvalue: 0.0293

2: drug effect on change in loss learning rate between baseline and week 2
group: 	beta: 0.32,	CI: [-0.05,0.69],	pvalue: 0.0878

3: loss learing rate related to anxiety over time
loss_LR: 	beta: 0.01,	CI: [0.0,0.02],	pvalue: 0.0185

4: change in aversive Pavlovian bias predicting treatment outcome (at week 12)
av_Pav_slope01: 	beta: 0.05,	CI: [0.0,0.09],	pvalue: 0.0312
av_Pav_slope01:group: 	beta: 0.07,	CI: [-0.02,0.15],	pvalue: 0.1197
----------------------------------------------------------------------------------------------------
placebo: av_Pav_slope01: 	beta: 0.01,	CI: [-0.05,0.07],	pvalue: 0.7912
sertraline: av_Pav_slope01: 	beta: 0.08,	CI: [0.02,0.14],	pvalue: 0.0089


In [63]:
# stats for whole sample
whole_sample_results=[]
m = gngstats.mle('loss_LR ~ group + time' + stratification_covariates, mle_df[mle_df['time']<2], ['group'],[])
whole_sample_results.append(m.summary().tables[1].iloc[1].values)
m = gngstats.glm('loss_LR_slope01 ~ group' + stratification_covariates, df_panda, ['group'])
whole_sample_results.append(convert_stats_table(m))
m = gngstats.glm('loss_LR_slope12 ~ group' + stratification_covariates, df_panda, ['group'])
whole_sample_results.append(convert_stats_table(m))
m = gngstats.mle('gadlog ~ loss_LR + group + time' + stratification_covariates, mle_df, ['loss_LR'], 'loss_LR')
whole_sample_results.append(m.summary().tables[1].iloc[1].values)
m = gngstats.glm('bdi3log ~ av_Pav_slope01 + group + bdi0log' + stratification_covariates, \
           df_panda, ['av_Pav_slope01'])
whole_sample_results.append(convert_stats_table(m))
m = gngstats.glm('bdi3log ~ av_Pav_slope01 * group + bdi0log' + stratification_covariates, \
           df_panda, ['av_Pav_slope01:group'])
whole_sample_results.append(convert_stats_table(m))


group: 	beta: 0.32,	CI: [0.03,0.61],	pvalue: 0.0293
group: 	beta: 0.32,	CI: [-0.05,0.69],	pvalue: 0.0878
group: 	beta: -0.35,	CI: [-0.74,0.04],	pvalue: 0.0801
loss_LR: 	beta: 0.01,	CI: [0.0,0.02],	pvalue: 0.0237
av_Pav_slope01: 	beta: 0.05,	CI: [0.0,0.09],	pvalue: 0.0312
av_Pav_slope01:group: 	beta: 0.07,	CI: [-0.02,0.15],	pvalue: 0.1197


In [64]:
from pylatex import Document, Package, Section, NoEscape
df_whole_sample_results = pd.DataFrame(whole_sample_results, columns = ['Coef.','Std.Err.','z','P>|z|','[0.025','0.975]'], \
             index=['drug effect on LR$^{loss}_{t2}$', 'drug effect on $\Delta$LR$^{loss}_{t2-t1}$', \
                   'drug effect on $\Delta$LR$^{loss}_{t3-t2}$', 'relation between GAD and LR$_{loss}$', \
                   'effect of $\Delta$Pav$^{aversive}_{t2-t1}$ on BDI$_{t4}$', \
                    'interaction effect of $\Delta$Pav$^{aversive}_{t2-t1}$*drug on BDI$_{t4}$'])
print(NoEscape(df_whole_sample_results.to_latex(escape=False)))

\begin{tabular}{lllllll}
\toprule
 & Coef. & Std.Err. & z & P>|z| & [0.025 & 0.975] \\
\midrule
drug effect on LR$^{loss}_{t2}$ & 0.320 & 0.147 & 2.180 & 0.029 & 0.032 & 0.607 \\
drug effect on $\Delta$LR$^{loss}_{t2-t1}$ & 0.322200 & 0.189000 & 1.707000 & 0.088000 & -0.048000 & 0.692000 \\
drug effect on $\Delta$LR$^{loss}_{t3-t2}$ & -0.349700 & 0.200000 & -1.750000 & 0.080000 & -0.741000 & 0.042000 \\
relation between GAD and LR$_{loss}$ & 0.009 & 0.004 & 2.261 & 0.024 & 0.001 & 0.016 \\
effect of $\Delta$Pav$^{aversive}_{t2-t1}$ on BDI$_{t4}$ & 0.045500 & 0.021000 & 2.154000 & 0.031000 & 0.004000 & 0.087000 \\
interaction effect of $\Delta$Pav$^{aversive}_{t2-t1}$*drug on BDI$_{t4}$ & 0.013800 & 0.029000 & 0.470000 & 0.639000 & -0.044000 & 0.071000 \\
\bottomrule
\end{tabular}



In [48]:
df_whole_sample_results

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
drug effect on LR$^{loss}_{t2}$,0.320,0.147,2.180,0.029,0.032,0.607
drug effect on $\Delta$LR$^{loss}_{t2-t1}$,0.3222,0.189,1.707,0.088,-0.048,0.692
drug effect on $\Delta$LR$^{loss}_{t3-t2}$,-0.3497,0.2,-1.75,0.08,-0.741,0.042
relation between GAD and LR$_{loss}$,0.009,0.004,2.261,0.024,0.001,0.016
effect of $\Delta$Pav$^{aversive}_{t2-t1}$ on BDI$_{t4}$,0.0455,0.021,2.154,0.031,0.004,0.087
interaction effect of $\Delta$Pav$^{aversive}_{t2-t1}$*drug on BDI$_{t4}$,0.0138,0.029,0.47,0.639,-0.044,0.071


Post-hoc testing findings in basic task characteristics:
--

In [49]:
# post-hoc analysis checking for switch probability
mle_df['switch_avoid'] = mle_df[['switch_g2a', 'switch_ng2a']].mean(axis=1)
mle_df['switch_avoid_zscore'] = gngstats.normalize(mle_df['switch_avoid'])
gngstats.mle('switch_avoid_zscore ~ group + time' + stratification_covariates + exclusion_covariates, \
           mle_df[(mle_df['time']<2)&(mle_df['exclusion']==0)], ['group'],[])

group: 	beta: 0.21,	CI: [0.0,0.41],	pvalue: 0.0482


In [50]:
gngstats.mle('gadlog ~ switch_avoid_zscore + group + time' + stratification_covariates + exclusion_covariates, \
           mle_df[mle_df['exclusion']==0], ['switch_avoid_zscore'], 'switch_avoid_zscore')

switch_avoid_zscore: 	beta: 0.0,	CI: [-0.02,0.02],	pvalue: 0.7371


In [51]:
p = 'ng2a_g2a'
for i in range(3):
    df_panda['ng2a_g2a' + str(i)] = df_panda['acc_ng2a' + str(i)] - df_panda['acc_g2a' + str(i)]
df_panda['ng2a_g2a_slope01'] = df_panda['ng2a_g2a1'] - df_panda['ng2a_g2a0']

j = 'bdi'
print('#' * 100)
print(j + ':')
gngstats.glm(j + '3log ~' + p + '_slope01 + ' + j + '0log + group' + stratification_covariates \
           + exclusion_covariates, df_panda[(df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
           [p + '_slope01'])
gngstats.glm(j + '3log ~' + p + '_slope01 * group +' +  j + '0log' + stratification_covariates \
           + exclusion_covariates, df_panda[(df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
           [p + '_slope01:group'])
print('-' * 100)
for i in range(2):
    print(group_label[i], end=': ')
    gngstats.glm(j + '3log ~' + p + '_slope01 +' +  j + '0log' + stratification_covariates \
               + exclusion_covariates, df_panda[(df_panda['group'] == i) & \
                                                (df_panda[['exclusion0', 'exclusion1']] == 0).all(axis=1)], \
               [p + '_slope01'])   

####################################################################################################
bdi:
ng2a_g2a_slope01: 	beta: 0.11,	CI: [-0.06,0.28],	pvalue: 0.2183
ng2a_g2a_slope01:group: 	beta: 0.34,	CI: [-0.01,0.68],	pvalue: 0.0563
----------------------------------------------------------------------------------------------------
placebo___: ng2a_g2a_slope01: 	beta: -0.0,	CI: [-0.22,0.22],	pvalue: 0.9841
sertraline: ng2a_g2a_slope01: 	beta: 0.27,	CI: [-0.01,0.56],	pvalue: 0.0606
